In [1]:
import pandas as pd
import numpy as np

In [2]:
pi_1 = [0,1,0,0]
o_states = ['S', 'A', 'B', 'C', 'D']
h_states = [1,2,3,4]
dim_h = 4
A_1_df = pd.DataFrame(columns=h_states, index=h_states)
A_1_df.loc[h_states[0]] = [1,0,0,0]
A_1_df.loc[h_states[1]] = [0,0,0,1]
A_1_df.loc[h_states[2]] = [0,0.4,0.3,0.3]
A_1_df.loc[h_states[3]] = [0.3,0.2,0.2,0.3]

B_1_df = pd.DataFrame(columns=o_states, index=h_states)
B_1_df.loc[h_states[0]] = [1,0,0,0,0]
B_1_df.loc[h_states[1]] = [0,0.5,0.5,0,0]
B_1_df.loc[h_states[2]] = [0,0.2,0.2,0.3,0.3]
B_1_df.loc[h_states[3]] = [0,0,0,0.5,0.5]


In [3]:
pi_2 = [0,0,0,1]
dim_h = 4
A_2_df = pd.DataFrame(columns=h_states, index=h_states)
A_2_df.loc[h_states[0]] = [1,0,0,0]
A_2_df.loc[h_states[1]] = [0.1,0.3,0.5,0.1]
A_2_df.loc[h_states[2]] = [0.1,0.4,0.3,0.2]
A_2_df.loc[h_states[3]] = [0.1,0.4,0.2,0.3]

B_2_df = pd.DataFrame(columns=o_states, index=h_states)
B_2_df.loc[h_states[0]] = [1,0,0,0,0]
B_2_df.loc[h_states[1]] = [0,0,0.5,0,0.5]
B_2_df.loc[h_states[2]] = [0,0,0.5,0.5,0]
B_2_df.loc[h_states[3]] = [0,0.5,0,0,0.5]


In [4]:
generated_sequence = []
hidden_states_sequence = []
gen_observations = []
hidden_states = []
number_states_map = {
    '1': 'S',
    '2': 'A',
    '3': 'B',
    '4': 'C',
    '5': 'D'
}

q1_index = np.random.choice(np.nonzero(pi_1)[0])
q1 = pi_1[q1_index]

def get_observation(q):
    observation_state = np.random.choice(np.nonzero(np.asarray(B_1_df.iloc[q]))[0])
    return observation_state, B_1_df.iloc[q][observation_state] == 1

def get_transition_state(q):
    return np.random.choice(np.nonzero(np.asarray(A_1_df.iloc[q]))[0])

q = q1
stop_state_reached = False
no_of_observations = 0

while no_of_observations <= 10:
    generated_sequence = []
    hidden_states_sequence = [q1+1]
    q = q1
    stop_state_reached = False
    
    while not stop_state_reached:
        o, stop_state_reached = get_observation(q)
        q = get_transition_state(q)
        generated_sequence.append(number_states_map[str(o+1)])
        hidden_states_sequence.append(q+1)
        
    gen_observations.append(generated_sequence)
    hidden_states.append(hidden_states_sequence)
    no_of_observations = no_of_observations + 1

print("Generated Sequences are::")
for obs in gen_observations:
    print(obs)

Generated Sequences are::
['B', 'D', 'S']
['A', 'D', 'A', 'D', 'S']
['A', 'D', 'B', 'C', 'C', 'B', 'D', 'B', 'C', 'B', 'C', 'S']
['B', 'D', 'S']
['B', 'D', 'C', 'S']
['B', 'C', 'S']
['A', 'D', 'B', 'C', 'D', 'A', 'B', 'B', 'D', 'B', 'D', 'A', 'D', 'B', 'D', 'D', 'C', 'S']
['B', 'C', 'D', 'S']
['A', 'C', 'B', 'C', 'A', 'C', 'D', 'D', 'C', 'B', 'D', 'D', 'C', 'B', 'D', 'B', 'C', 'S']
['A', 'C', 'D', 'B', 'C', 'D', 'S']
['A', 'D', 'D', 'A', 'D', 'S']


In [5]:
def hmm_forward(transition_prob_matrix, emission_matrix, priors, obs_seq):
    T = len(obs_seq)
    N = len(priors)
    
    alpha = np.zeros((N, T))
    alpha[:,0] = priors * emission_matrix[obs_seq[0]]

    for t in range(1,T):
        for j in range(N):
            for i in range(N):
                alpha[j, t] += transition_prob_matrix.values[i][j] * emission_matrix.iloc[j][obs_seq[t]] * alpha[i][t-1]
                
    return alpha, np.sum(alpha[:,T-1])

In [6]:
observations = [
    ['A','D','C','B','D','C','C','S'],
    ['B','D','S'],
    ['B','C','C','B','D','D','C','A','C','S'],
    ['A','C','D','S'],
    ['A','D','A','C','S'],
    ['D','B','B','S'],
    ['A','B','S'],
    ['D','D','B','D','D','B','A','C','C','D','A','B','B','C','D','B','B','B','S'],
    ['D','B','D','S'],
    ['A','A','A','A','D','C','B','S'],
]

for obs in observations:
    p1, p_lambda1 = hmm_forward(A_1_df, B_1_df, pi_1, obs)
    p2, p_lambda2 = hmm_forward(A_2_df, B_2_df, pi_2, obs)
    print(p_lambda1, p_lambda2)
    if(p_lambda1 > p_lambda2):
        print("lambda 1")
    else:
        print("lambda_2")

3.762855e-05 8.53125e-06
lambda 1
0.075 0.0
lambda 1
3.98671632e-06 0.0
lambda 1
0.01125 0.0015000000000000005
lambda 1
0.0042 0.00016250000000000002
lambda 1
0.0 0.005750000000000001
lambda_2
0.0 0.015000000000000003
lambda_2
0.0 1.4033509570312503e-12
lambda_2
0.0 0.0035000000000000005
lambda_2
0.0 3.8390625e-06
lambda_2


## For testing sequences generated:

In [7]:
for obs in gen_observations:
    p1, p_lambda1 = hmm_forward(A_1_df, B_1_df, pi_1, obs)
    p2, p_lambda2 = hmm_forward(A_2_df, B_2_df, pi_2, obs)
    print(p_lambda1, p_lambda2)
    if(p_lambda1 > p_lambda2):
        print("lambda 1")
    else:
        print("lambda_2")

0.075 0.0
lambda 1
0.0042 0.00056875
lambda 1
2.648882184120001e-07 8.242910156250002e-08
lambda 1
0.075 0.0
lambda 1
0.01125 0.0
lambda 1
0.075 0.0
lambda 1
2.3236294660771974e-12 1.0746817382812503e-11
lambda_2
0.01125 0.0
lambda 1
4.60433887360169e-11 1.3904707031250003e-12
lambda 1
0.000177795 3.225000000000001e-05
lambda 1
0.0011205 0.00013781250000000002
lambda 1


## Viterbi Algorithm

HMM Viterbi algorithm to identify hidden states given the observation sequence

In [8]:
def hmm_viterbi(transition_prob_matrix, emission_matrix, priors, observations):
    T = len(observations)
    N = len(priors)
    
    delta = np.zeros((T, N))
    xi = np.zeros((T, N), 'int')
    
    delta[0, :] = priors * emission_matrix[observations[0]]
    
    for t in range(1, T):
        for j in range(N):
            maxP = -999
            idx = -1
            for i in range(N):
                temp = delta[t-1][i] * transition_prob_matrix.values[i][j] * emission_matrix.iloc[j][observations[t]]
                if temp > maxP:
                    maxP = temp
                    idx = i
                delta[t][j] = maxP
                xi[t][j] = idx
    
    state_seq = []
    state_seq.append(np.argmax(delta[T-1,:]))
    for i in range(T-1, 0, -1):
        state_seq.append(xi[i, state_seq[-1]])
    state_seq = state_seq[::-1]
    for i in range(len(state_seq)):
        state_seq[i] = state_seq[i]+1
        
    return state_seq

In [9]:
observations = [
    ['A','D','C','B','D','C','C','S'],
    ['B','D','S'],
    ['B','C','C','B','D','D','C','A','C','S'],
    ['A','C','D','S'],
    ['A','D','A','C','S'],
    ['D','B','B','S'],
    ['A','B','S'],
    ['D','D','B','D','D','B','A','C','C','D','A','B','B','C','D','B','B','B','S'],
    ['D','B','D','S'],
    ['A','A','A','A','D','C','B','S'],
]

for obs in observations:
    print(obs)
    print("Hidden states sequence::")
    print(hmm_viterbi(A_2_df,B_2_df,pi_2,obs))
    print("-------------------------------------")

['A', 'D', 'C', 'B', 'D', 'C', 'C', 'S']
Hidden states sequence::
[4, 2, 3, 2, 2, 3, 3, 1]
-------------------------------------
['B', 'D', 'S']
Hidden states sequence::
[1, 1, 1]
-------------------------------------
['B', 'C', 'C', 'B', 'D', 'D', 'C', 'A', 'C', 'S']
Hidden states sequence::
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
-------------------------------------
['A', 'C', 'D', 'S']
Hidden states sequence::
[4, 3, 2, 1]
-------------------------------------
['A', 'D', 'A', 'C', 'S']
Hidden states sequence::
[4, 4, 4, 3, 1]
-------------------------------------
['D', 'B', 'B', 'S']
Hidden states sequence::
[4, 2, 3, 1]
-------------------------------------
['A', 'B', 'S']
Hidden states sequence::
[4, 2, 1]
-------------------------------------
['D', 'D', 'B', 'D', 'D', 'B', 'A', 'C', 'C', 'D', 'A', 'B', 'B', 'C', 'D', 'B', 'B', 'B', 'S']
Hidden states sequence::
[4, 2, 3, 2, 2, 3, 4, 3, 3, 4, 4, 2, 2, 3, 2, 3, 2, 3, 1]
-------------------------------------
['D', 'B', 'D', 'S']
Hidden stat